## Notebook to create brain state files from EF1_ALPHA excel sleep scoring spreadsheet

In [1]:
import os 
import numpy as np
import pandas as pd

In [403]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA')
epoch_number = "A, E, I, M, Q, U, Y, AC, AG, AK, AO, AS, BA, BE, BI, BM, BQ, BU, BY, CC, CG, CK, CO, CS, CW"
cols = "B, F, J, N, R, V, Z, AD, AH, AL, AP, AT, BB, BF, BJ, BN, BR, BV, BZ,CD, CH, CL, CP, CT, CX"
cols_discard = "C, G, K, O, S, W, AA, AE, AI, AM, AQ, AU, BC, BG, BK, BO, BS, BW, CA, CE, CI, CM, CQ, CU, CY"
skiprows = 5
raw_excel = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header= 1)

In [317]:
#list of animal ids
column_names = list(raw_excel)
animal_ids = [anim_id[0:8] for anim_id in column_names]

In [319]:
len(animal_ids)

25

In [404]:
raw_epochs = pd.read_excel('grant_EEG_classification.xlsx', usecols = epoch_number, header = 1, skiprows = 6)
raw_epochs_letter = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header = 1, skiprows = 6)
raw_discard = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols_discard, header = 1, skiprows = 6)

In [420]:
two_files = []
for epoch in raw_epochs:
    start_epochs = raw_epochs[raw_epochs[epoch] == 0].index.tolist()
    two_files.append(start_epochs)

In [438]:
two_files

[[0, 912],
 [0, 2391],
 [0, 2442],
 [0],
 [0, 1130],
 [0, 1476],
 [0, 2442],
 [0],
 [0, 2442],
 [0],
 [0],
 [0, 1521],
 [0],
 [0],
 [0],
 [0, 912],
 [0, 2391],
 [0],
 [0, 1521],
 [0],
 [0, 1521],
 [0],
 [0],
 [0, 2442],
 [0]]

In [475]:
for file, epoch, animal_id, epoch_letter_col, discard_col in zip(two_files, raw_epochs, animal_ids, raw_epochs_letter, raw_discard):
    if len(file) == 2:
        start_2 = file[1:2]
        start_2 = start_2[0]
        end_epoch = start_2 - 1
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        part_1 = epoch_column[0: end_epoch]
        part_2 = epoch_column[start_2: length_col]
        epochs_letter_part_1 = epochs_letter[0:end_epoch]
        epochs_letter_part_2 = epochs_letter[start_2: length_col]
        discard_data = raw_discard[discard_col]
        discard_part_1 = discard_data[0: end_epoch]
        discard_part_2 = discard_data[start_2: length_col]
        part_1_dict = {'epoch_numbers': part_1, 'brain_state': epochs_letter_part_1,
                       'epoch_discard_numbers': discard_part_1}
        part_2_dict = {'epoch_numbers': part_2, 'brain_state': epochs_letter_part_2,
                      'epoch_discard_numbers': discard_part_2}
        df_1 = pd.DataFrame(data = part_1_dict)
        df_2 = pd.DataFrame(data = part_2_dict)
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        df_1.to_csv(str(animal_id) +'_part_1.csv', index=True) 
        df_2.to_csv(str(animal_id) +'_part_2.csv', index=True)
    else:
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        all_epochs = epoch_column_raw[0:length_col]
        all_epochs_letter = epochs_letter[0:length_col]
        discard_data = raw_discard[discard_col]
        all_discard_epochs = discard_data[0:length_col] 
        data_dict = {'epoch_numbers': all_epochs, 'brain_state': all_epochs_letter,
                       'epoch_discard_numbers': all_discard_epochs}
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        data_df = pd.DataFrame(data = data_dict)
        data_df.to_csv(str(animal_id) +'.csv', index=True)

In [476]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
test_check = pd.read_csv('191216_B.csv')
test_check

,Unnamed: 0,epoch_numbers,brain_state,epoch_discard_numbers
0,0,0.0,W,NaN
1,1,1.0,W,NaN
2,2,2.0,W,NaN
3,3,3.0,W,NaN
4,4,4.0,W,NaN
...,...,...,...,...
2784,2784,2784.0,N,E
2785,2785,2785.0,N,E
2786,2786,2786.0,N,E
2787,2787,2787.0,N,E


In [466]:
wake_indices = test_check.loc[test_check['brain_state'] == 'W']
discard_indices = test_check.loc[test_check['epoch_discard_numbers'] == 'E']

In [477]:
wake_indices_list = wake_indices.index.tolist()
discard_indices_list = discard_indices.index.tolist()

In [395]:
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match

In [400]:
def match_elements(list_a, list_b):
    match = []
    for i in list_a:
        if i in list_b:
            match.append(i)
    return match

In [468]:
new_list = non_match_elements(wake_indices_list, discard_indices_list)

In [469]:
check_list = match_elements(new_list, discard_indices_list)
check_list

[]

In [363]:
def load_brainstate_file(brainstate_file, brainstate_letter):
        brainstate_indices = brainstate_file.loc[brainstate_file['brain_state'] == brainstate_letter]
        discard_indices = brainstate_file.loc[brainstate_file['epoch_discard'] == 'E']
        
        all_indices = brainstate_indices.index
        starting_index = all_indices[0]
            
        epoch_indices = []
        
        for epoch_index in range(len(all_indices)-1):
            if all_indices[epoch_index] + 1 != all_indices[epoch_index + 1]:
                epoch_indices.append([starting_index, all_indices[epoch_index]])
                starting_index = all_indices[epoch_index + 1]

        #append last value outside of the loop as the loop is for len -1
        epoch_indices.append([starting_index, all_indices[-1]])

        return epoch_indices
